In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Cleaning

In [3]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [4]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,...,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,...,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,...,0,0,0,0,0,1,0,0,0,0
8,66310712.0,71035433.0,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,1,0,0,0,0
9,68476807.0,73366655.0,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

# Remove id and member id
y2015 = y2015[y2015.columns[2:]]

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y =y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.98067013, 0.98048016, 0.98197621, 0.98169124, 0.97041083,
       0.97696509, 0.96810658, 0.9808592 , 0.97978958, 0.98073909])

In [12]:
# Create correlation matrix with the remaining columns
correlation_matrix = X.corr()
display(correlation_matrix)

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
loan_amnt,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,-0.039040,0.188214,...,0.003849,-0.000398,0.009984,0.000568,-0.004610,-0.016880,0.016880,0.008359,0.007089,0.014350
funded_amnt,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,-0.039040,0.188214,...,0.003849,-0.000398,0.009984,0.000568,-0.004610,-0.016880,0.016880,0.008359,0.007089,0.014350
funded_amnt_inv,0.999994,0.999994,1.000000,0.140209,0.941187,0.305803,0.006044,-0.010448,-0.039103,0.188212,...,0.003850,-0.000394,0.009988,0.000578,-0.004605,-0.016832,0.016832,0.008338,0.007071,0.014305
int_rate,0.140572,0.140572,0.140209,1.000000,0.124426,-0.090399,0.077932,0.043710,0.231139,-0.012985,...,0.012297,0.007406,0.078885,0.004256,0.013182,-0.020755,0.020755,0.008172,0.011635,0.018622
installment,0.941205,0.941205,0.941187,0.124426,1.000000,0.297393,0.001893,-0.002129,-0.006255,0.169995,...,0.006806,0.001181,0.018691,0.002285,-0.002827,-0.015129,0.015129,0.006155,0.006336,0.014613
annual_inc,0.305734,0.305734,0.305803,-0.090399,0.297393,1.000000,-0.068237,0.037910,0.022990,0.122328,...,-0.001767,-0.001858,-0.005633,0.001250,-0.003269,0.008627,-0.008627,-0.003188,-0.005930,-0.007350
dti,0.006103,0.006103,0.006044,0.077932,0.001893,-0.068237,1.000000,-0.005868,0.003277,0.104364,...,-0.001939,-0.003252,0.002700,-0.004712,-0.001814,-0.077534,0.077534,0.086092,0.011906,0.016317
delinq_2yrs,-0.010424,-0.010424,-0.010448,0.043710,-0.002129,0.037910,-0.005868,1.000000,0.036139,0.040005,...,-0.002120,-0.002694,-0.000831,-0.002338,-0.003097,-0.000550,0.000550,0.001747,0.001023,-0.001930
inq_last_6mths,-0.039040,-0.039040,-0.039103,0.231139,-0.006255,0.022990,0.003277,0.036139,1.000000,0.159144,...,0.008670,0.002087,0.032469,0.000325,0.015494,-0.001600,0.001600,0.000415,0.000925,0.001699
open_acc,0.188214,0.188214,0.188212,-0.012985,0.169995,0.122328,0.104364,0.040005,0.159144,1.000000,...,-0.001515,-0.005219,-0.004271,-0.006863,0.005028,0.005119,-0.005119,-0.005620,-0.001326,-0.000835


In [16]:
y2 = y2015.columns.is_numeric

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=4)
X = StandardScaler().fit_transform(y2)
principalComponents = pca.fit_transform(y2)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2',
                          'principal component 3', 'principal component 4'])
principalDf.head()

TypeError: float() argument must be a string or a number, not 'method'